# 使用Julia调用dll

Julia官方文档地址：[Calling C and Fortran Code](https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/)

在开始前，请务必确认Julia的位数与所用dll位数相同，否则会报错dll不是一个可用的Win32应用。

ForDllCreate.dll与ForDllCreate.64.dll内部包含和前文相同的函数，区别是前者是32位，后者是64位。
在64位REPL上载入32位dll会报错

ERROR: LoadError: could not load library "e:\yyb\HF_first\ForDllCreat.dll"
%1 is not a valid Win32 application.
Stacktrace:
 [1] top-level scope
   @ e:\yyb\HF_first\test.jl:15
in expression starting at e:\yyb\HF_first\test.jl:15

先贴上错误示例。

In [ ]:
//error
a=[1]
b=[2]
c=[0]
ccall((:OUTPUT, ".\\ForDllCreat.dll"),Cvoid,(Ptr{Cint},Ptr{Cint},Ptr{Cint}),pointer_(a),pointer(b),pointer(c))
print(c)

In [ ]:
a=[1]
b=[2]
c=[0]
ccall((:OUTPUT, ".\\ForDllCreat64.dll"),Cvoid,(Ptr{Cint},Ptr{Cint},Ptr{Cint}),pointer_from_objref(a)+0x40,pointer_from_objref(b)+0x40,pointer_from_objref(c)+0x40)
print(c)

Julia可用通过ccall函数调用C和Fortran编译的dll文件，输入格式为

```txt
  ccall((function_name, library), returntype, (argtype1, ...), argvalue1, ...)
  ccall(function_name, returntype, (argtype1, ...), argvalue1, ...)  
  ccall(function_pointer, returntype, (argtype1, ...), argvalue1, ...)
```

这里通过第一种调用方法来调用我们编译的ForDllCreat64.dll，

function_name是调用的函数名称。引用时即可以用:OUTPUT表示，也可以用"OUTPUT"表示。

C语言与Fortran输出dll时函数名不变，C++输出函数有命名粉碎，自制dll尽量采用C输出，一定要确定被调用函数的名字才能成功引用。[可看此视频](https://www.bilibili.com/video/BV1dW411G7hL)

library是被调用dll的路径，用字符串表示。调用C标准库中的函数时，library可以略去。

In [ ]:
#调用C标准库函数，不用写引用
t = ccall(:clock, Int32, ())

returntype是被调函数的返回类型。Fortran的subroutine返回类型是空，即void，在Julia中表示为Cvoid。数据类型对应的表格可以参考下文表格，也可以查看官方文档。


(argtype1, ...)是一个tuple，与被调函数的输入变量类型要一一对应，类似在C++中定义一个与被调函数输入变量类型一一对应的函数原型。

argvalue1, ...  这部分是输入变量，类型要与(argtype1, ...)一一对应，并与被调函数对应。输入变量不用tuple表示。

不同语言间调用dll，最重要的就是数据类型的匹配。下表是从Julia文档中复制的数据类型对应表。

<table><tbody><tr><th style="text-align: left">C name</th><th style="text-align: left">Fortran name</th><th style="text-align: left">Standard Julia Alias</th><th style="text-align: left">Julia Base Type</th></tr><tr><td style="text-align: left"><code>unsigned char</code></td><td style="text-align: left"><code>CHARACTER</code></td><td style="text-align: left"><code>Cuchar</code></td><td style="text-align: left"><code>UInt8</code></td></tr><tr><td style="text-align: left"><code>bool</code> (_Bool in C99+)</td><td style="text-align: left"></td><td style="text-align: left"><code>Cuchar</code></td><td style="text-align: left"><code>UInt8</code></td></tr><tr><td style="text-align: left"><code>short</code></td><td style="text-align: left"><code>INTEGER*2</code>, <code>LOGICAL*2</code></td><td style="text-align: left"><code>Cshort</code></td><td style="text-align: left"><code>Int16</code></td></tr><tr><td style="text-align: left"><code>unsigned short</code></td><td style="text-align: left"></td><td style="text-align: left"><code>Cushort</code></td><td style="text-align: left"><code>UInt16</code></td></tr><tr><td style="text-align: left"><code>int</code>, <code>BOOL</code> (C, typical)</td><td style="text-align: left"><code>INTEGER*4</code>, <code>LOGICAL*4</code></td><td style="text-align: left"><code>Cint</code></td><td style="text-align: left"><code>Int32</code></td></tr><tr><td style="text-align: left"><code>unsigned int</code></td><td style="text-align: left"></td><td style="text-align: left"><code>Cuint</code></td><td style="text-align: left"><code>UInt32</code></td></tr><tr><td style="text-align: left"><code>long long</code></td><td style="text-align: left"><code>INTEGER*8</code>, <code>LOGICAL*8</code></td><td style="text-align: left"><code>Clonglong</code></td><td style="text-align: left"><code>Int64</code></td></tr><tr><td style="text-align: left"><code>unsigned long long</code></td><td style="text-align: left"></td><td style="text-align: left"><code>Culonglong</code></td><td style="text-align: left"><code>UInt64</code></td></tr><tr><td style="text-align: left"><code>intmax_t</code></td><td style="text-align: left"></td><td style="text-align: left"><code>Cintmax_t</code></td><td style="text-align: left"><code>Int64</code></td></tr><tr><td style="text-align: left"><code>uintmax_t</code></td><td style="text-align: left"></td><td style="text-align: left"><code>Cuintmax_t</code></td><td style="text-align: left"><code>UInt64</code></td></tr><tr><td style="text-align: left"><code>float</code></td><td style="text-align: left"><code>REAL*4i</code></td><td style="text-align: left"><code>Cfloat</code></td><td style="text-align: left"><code>Float32</code></td></tr><tr><td style="text-align: left"><code>double</code></td><td style="text-align: left"><code>REAL*8</code></td><td style="text-align: left"><code>Cdouble</code></td><td style="text-align: left"><code>Float64</code></td></tr><tr><td style="text-align: left"><code>complex float</code></td><td style="text-align: left"><code>COMPLEX*8</code></td><td style="text-align: left"><code>ComplexF32</code></td><td style="text-align: left"><code>Complex{Float32}</code></td></tr><tr><td style="text-align: left"><code>complex double</code></td><td style="text-align: left"><code>COMPLEX*16</code></td><td style="text-align: left"><code>ComplexF64</code></td><td style="text-align: left"><code>Complex{Float64}</code></td></tr><tr><td style="text-align: left"><code>ptrdiff_t</code></td><td style="text-align: left"></td><td style="text-align: left"><code>Cptrdiff_t</code></td><td style="text-align: left"><code>Int</code></td></tr><tr><td style="text-align: left"><code>ssize_t</code></td><td style="text-align: left"></td><td style="text-align: left"><code>Cssize_t</code></td><td style="text-align: left"><code>Int</code></td></tr><tr><td style="text-align: left"><code>size_t</code></td><td style="text-align: left"></td><td style="text-align: left"><code>Csize_t</code></td><td style="text-align: left"><code>UInt</code></td></tr><tr><td style="text-align: left"><code>void</code></td><td style="text-align: left"></td><td style="text-align: left"></td><td style="text-align: left"><code>Cvoid</code></td></tr><tr><td style="text-align: left"><code>void</code> and <code>[[noreturn]]</code> or <code>_Noreturn</code></td><td style="text-align: left"></td><td style="text-align: left"></td><td style="text-align: left"><code>Union{}</code></td></tr><tr><td style="text-align: left"><code>void*</code></td><td style="text-align: left"></td><td style="text-align: left"></td><td style="text-align: left"><code>Ptr{Cvoid}</code> (or similarly <code>Ref{Cvoid}</code>)</td></tr><tr><td style="text-align: left"><code>T*</code> (where T represents an appropriately defined type)</td><td style="text-align: left"></td><td style="text-align: left"></td><td style="text-align: left"><code>Ref{T}</code> (T may be safely mutated only if T is an isbits type)</td></tr><tr><td style="text-align: left"><code>char*</code> (or <code>char[]</code>, e.g. a string)</td><td style="text-align: left"><code>CHARACTER*N</code></td><td style="text-align: left"></td><td style="text-align: left"><code>Cstring</code> if NUL-terminated, or <code>Ptr{UInt8}</code> if not</td></tr><tr><td style="text-align: left"><code>char**</code> (or <code>*char[]</code>)</td><td style="text-align: left"></td><td style="text-align: left"></td><td style="text-align: left"><code>Ptr{Ptr{UInt8}}</code></td></tr><tr><td style="text-align: left"><code>jl_value_t*</code> (any Julia Type)</td><td style="text-align: left"></td><td style="text-align: left"></td><td style="text-align: left"><code>Any</code></td></tr><tr><td style="text-align: left"><code>jl_value_t* const*</code> (a reference to a Julia value)</td><td style="text-align: left"></td><td style="text-align: left"></td><td style="text-align: left"><code>Ref{Any}</code> (const, since mutation would require a write barrier, which is not possible to insert correctly)</td></tr><tr><td style="text-align: left"><code>va_arg</code></td><td style="text-align: left"></td><td style="text-align: left"></td><td style="text-align: left">Not supported</td></tr><tr><td style="text-align: left"><code>...</code> (variadic function specification)</td><td style="text-align: left"></td><td style="text-align: left"></td><td style="text-align: left"><code>T...</code> (where <code>T</code> is one of the above types, when using the <code>ccall</code> function)</td></tr><tr><td style="text-align: left"><code>...</code> (variadic function specification)</td><td style="text-align: left"></td><td style="text-align: left"></td><td style="text-align: left"><code>; va_arg1::T, va_arg2::S, etc.</code> (only supported with <code>@ccall</code> macro)</td></tr></tbody></table>

# Julia中指针的用法

**此处有不明点，请实践时小心处理**

数组类型基本上通过指针传递。

Julia中，指针有两种，Ptr{T}与Ref{T}

Ptr表示的是从变量获得的地址，这类地址是否被销毁不由Julia管理，一般是“危险的”（unsafe)。

Ref是由Julia分配的地址，这类地址的任何更改都由Julia进行，因此是“安全的”。

但是Ref能用的方法似乎不多，目前为止我没学明白这个怎么用。

Julia中获得变量地址的函数有pointer和pointer_form_objref，他们获得的指针都是Ptr型的
pointer获得的地址被标明了数据类型，并且总比pointer_form_objref的返回值多出一个数据类型的bit数；pointer_form_objref获得的地址是无数据类型的。
在官方文档中，pointer_from_objref是对C提供接口的方法([C_Interface](https://docs.julialang.org/en/v1/base/c/))

In [ ]:
a="大家好"
b=pointer(a)
c=pointer_from_objref(a)
println(b)
println(c)
println(b-c)

println("a[1]的字节数是",sizeof(typeof(a[1])))#UInt8的字节数是1,但是Char类型的字节数是4

由地址获得值的方法是unsafe_pointer_to_objref，这个函数也是官方文档中C接口的函数。

对一个Ptr指针，用pointer获得的指针要减去一个数据类型的bit数才能获得指针内的值。

In [ ]:
a=[1.23]

b=pointer(a)
c=pointer_from_objref(a)

d=unsafe_pointer_to_objref(b-0x40)
e=unsafe_pointer_to_objref(c)

println("d=",d)
println("e=",e)
